In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=C:/Users/cherp/AppData/Local/Google/Chrome/User Data/Default')

driver = webdriver.Chrome(options=options)

page_number = 1

router_link_uris = []

while True:
    url = f"https://megamarket.ru/catalog/smartfony/page-{page_number}"
    driver.get(url)
    
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[contains(text(),'Товары не в наличии')]")))
        break
    except:
        pass
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    catalog_items_div = soup.find('div', class_='catalog-listing__items catalog-listing__items_divider')
    
    if catalog_items_div:
        div_id_blocks = catalog_items_div.find_all('div', id=True)
        for div_id_block in div_id_blocks:
            router_link_uri = div_id_block['router-link-uri']
            router_link_uris.append(router_link_uri)
    
    page_number += 1
    print(len(router_link_uris))

driver.quit()

for link_uri in router_link_uris:
    print(link_uri)


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=C:/Users/cherp/AppData/Local/Google/Chrome/User Data/Default')

driver = webdriver.Chrome(options=options)

data = []

for router_link_uri in tqdm(router_link_uris):
    full_url = f"https://megamarket.ru{router_link_uri}"
    
    try:
        driver.get(full_url)
        
        pdp_tab_selector = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'pdp-tab-selector')))
        
        links = pdp_tab_selector.find_elements(By.TAG_NAME, 'a')
        
        for link in links:
            href = link.get_attribute("href")
            if "details_block=prices" in href:
                WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CLASS_NAME, 'flocktory-widget-overlay')))
                
                link.click()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'product-offers')))
                
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, 'html.parser')
                product_offers_div = soup.find('div', {'class': 'product-offers product-offers_no-border pdp-prices__offers'})
                
                if product_offers_div:
                    product_offer_divs = product_offers_div.find_all('div', class_='product-offer')
                    
                    pairs_data = []
                    
                    for product_offer_div in product_offer_divs:
                        product_offer_name_div = product_offer_div.find('div', class_='product-offer-name')
                        
                        product_price_span = product_offer_div.find('span', class_='product-offer-price__amount')
                        
                        if product_offer_name_div and product_price_span:
                            organization_text = product_offer_name_div.text.strip()
                            price_text = product_price_span.text.strip()

                            # Используем регулярное выражение для удаления текста после "Оплата только наличными" или "Оплата только онлайн"
                            organization = re.sub(r'(Оплата только наличными|Оплата только онлайн).*', '', organization_text).strip()
        
                            pairs_data.extend([organization, price_text])
                    
                    # Добавляем данные в формате "Ссылка, Название1, Цена1, Название2, Цена2, ..." в список
                    data.append([full_url] + pairs_data)
    
    except Exception as e:
        print(f"Ошибка: {e}")
        continue

driver.quit()

df = pd.DataFrame(data)

df.to_excel("product_data9.xlsx", index=False)